In [156]:
from datetime import date
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

In [157]:
df = pd.read_csv("../Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.Ticker('BTC-USD').history(period='7y',interval='1d',actions=False).reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Open','High','Low', 'Volume', 'Unnamed: 0', "wallets", "address", "mined"])
df['200D'] = df['price'].rolling(200).mean()
df['300D'] = df['price'].rolling(300).mean()
df['50D'] = df['price'].rolling(50).mean()
df['7D'] = df['price'].rolling(7).mean()
# df['200W'] = df['price'].rolling(1400).mean()
df = df.dropna()
df['meanavge'] = (df['200D'] + df['300D'] + df['50D'] )/3
# df = df.drop(columns=['200D','300D', '50D'])
df['meanvalue'] = df["price"] - df["meanavge"]
df['status'] = df['meanvalue'].apply(lambda x: '1' if x > 0 else '0')
df['status']=df['status'].astype("object")
df['price-meanavge']=df['price'] - df['meanavge']
df['move%'] = df['price-meanavge']/(df['price'] + df['meanavge'])
bins = [-0.43, -0.18, 0, 0.18, 0.43]
group_names = ["Severely Oversold","Neutral Oversold", "Neutral Overbought","Severely Overbought"]
df["Valuation"] = pd.cut(df["move%"], bins, labels=group_names)
pricefrommean = df.meanvalue.iloc[-1].round(2)
currentzone = df.Valuation.iloc[-1]
delta1cycle = df.index[df['date']=='2015-09-15'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2cycle = df.index[df['date']=='2019-04-2'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averageunder = int((delta1cycle+delta2cycle)/2)
delta1fromp2p = df.index[df['date']=='2017-03-17'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2fromp2p = df.index[df['date']=='2020-11-29'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averagep2p = int((delta1fromp2p+delta2fromp2p)/2)
sincealltimehigh = df.index[-1] - df.index[df['price']==df.price.max()].tolist()[0]

In [158]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.price.iloc[-1],
    mode = "gauge+number+delta",
    title = {'text': "Current Price and ATH Delta"},
    number = {'prefix': "$"},
    delta = {'reference': df.price[df['price']==df.price.max()].tolist()[0], "valueformat": ".0f"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

fig.show()
fig.write_html("../static/priceath.html")

In [159]:
import plotly.graph_objects as go


fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.meanavge.iloc[-1],
    mode = "number+delta",
    number = {'prefix': "$"},
    delta = {"reference": ((df.meanavge.iloc[-2])), "valueformat": ".0f"},
    title = {'text': "Meanaverage Day Change"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))


fig.show()
fig.write_html("../static/meanaverage.html")

In [160]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.price.iloc[-1] - df.meanavge.iloc[-1],
    mode = "number+delta",
    number = {'prefix': "$"},
    delta = {"reference": ((df.price.iloc[-2] - df.meanavge.iloc[-2])), "valueformat": ".0f"},
    title = {'text': "Current Price Meanaverage Delta Day Change"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

fig.show()
fig.write_html("../static/price-mean.html")

In [161]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    title = {'text': "Current Valuation - Standard Deviation percent move from meanaverage", 'font': {'size': 15}},
    value = df['move%'].iloc[-1],
    domain = {'x': [0,1], 'y': [0,1]},
    gauge = {
        'axis': {'range': [-0.43, 0.43], 'tickwidth': 1, 'tickcolor': "orange"},
        'bar': {'color': "orange"},
        'bgcolor': "white",
        'borderwidth': 2,
        'bordercolor': "white",
        'steps': [
            {'range': [-0.43, -0.18], 'color': 'red'},
            {'range': [-0.18, 0], 'color': 'yellow'},
            {'range': [0, 0.18], 'color': 'blue'},
            {'range': [0.18,0.43], 'color': 'green'}],
        }))

fig.update_layout(
    font={'color': "black", 'family': "Arial"},
    xaxis={'showgrid': False, 'range':[-1,1]},
    yaxis={'showgrid': False, 'range':[0,1.11]},
    plot_bgcolor='rgba(0,0,0,0)'
    )


# Set the visibility ON
fig.update_yaxes(title='', visible=True, showticklabels=False)
# Set the visibility OFF
fig.update_xaxes(title='x', visible=False, showticklabels=False)

fig.show()
fig.write_html("../static/valuation.html")

In [162]:
import yfinance as yf
import pandas as pd

def get_ytd_returns(ticker):
    df = yf.Ticker(ticker).history(period='ytd',interval='1d',actions=False).reset_index()
    ytd_return = ((df.Close.iloc[-1]/df.Close.iloc[0])-1)
    return ytd_return

tickers = ['BTC-USD', 'GC=F', 'Cl=F', 'TLT', 'DX=F', '^RUT', '^GSPC', '^IXIC', '^DJI', 'BIGPX']
asset_names = ['Bitcoin', 'Gold', 'Oil', '20yr Treasury', 'DXY', 'Russell 2000 Index', 'S&P 500', 'NASDAQ', 'Dow Jones', '60/40']

data = [[asset_name, get_ytd_returns(ticker)*100] for asset_name, ticker in zip(asset_names, tickers)]

# Create the pandas DataFrame
data = pd.DataFrame(data, columns=['Asset', 'YTD%'])
data['YTD%'] = data['YTD%'].round(2)


In [163]:
import plotly.express as px
fig = px.bar(data, x='YTD%', y='Asset', orientation='h', text=str("YTD%"), color="YTD%",color_continuous_scale=px.colors.sequential.Oryel)
fig.layout.yaxis.tickformat = ',.%'
fig.update_layout(title_text='Year To Date Returns')
fig.update_layout(yaxis=dict(showticklabels=True))

fig.update_layout(
    margin=dict(l=4, r=100, t=70, b=20),
)

fig.write_html("../static/YTD.html")
fig.show()

In [164]:
import pandas as pd
import yfinance as yf
from datetime import timedelta, date

# Set the start and end date
start_date = date.today() - timedelta(days=30)
end_date = date.today() 

# Define the ticker list and add BTC-USD to it
# tickers_list = ['GC=F', 'Cl=F', 'TLT', 'DX=F', '^RUT', '^GSPC', '^IXIC', '^DJI', 'BIGPX']
tickers_list = [ 'DX=F', '^GSPC', '^IXIC','GC=F']
tickers_list.insert(0, 'BTC-USD')

# Fetch the data and drop rows with missing values
# data2 = yf.Ticker(tickers_list).history(period='3mo',interval='1d',actions=False)['Close'].dropna()
data1 = yf.download(tickers_list, start=start_date, end=end_date)['Close'].dropna()

# Compute the rolling correlation between BTC-USD and other tickers and add the columns to the data frame
corr_cols = [col.replace(tickers_list[0], 'BTC') for col in tickers_list[1:]]
for i, col in enumerate(corr_cols):
    corr = data1.iloc[:, 0].rolling(3).corr(data1.iloc[:, i+1])
    data1[col] = corr.values

# Reset the index
data1 = data1.reset_index()

col_mapping = {
    # "BIGPX": "60/40",
    # "TLT": "20 Yr Treasury",
    # "Cl=F": "Oil",
    "GC=F": "Gold",
    # "^DJI": "Dow Jones",
    "^GSPC": "S&P 500",
    "^IXIC": "Nasdaq",
    "DX=F": "DXY"
    # "^RUT": "Russ 2000"
}

# Rename the columns using the mapping
data1 = data1.rename(columns=col_mapping)
data1=data1.drop(columns=['BTC-USD']).dropna()
data1

[*********************100%***********************]  5 of 5 completed


,Date,DXY,Gold,S&P 500,Nasdaq
6,2023-02-27,-0.890024,-0.119325,0.877016,-0.590427
7,2023-02-28,-0.703637,-0.447068,-0.228770,0.578351
8,2023-03-01,-0.963715,-0.271698,0.153316,0.589562
9,2023-03-02,-0.538581,-0.092965,0.988255,0.472072
10,2023-03-03,0.305230,-0.903812,-0.915106,0.838697
11,2023-03-06,0.958474,-0.939193,-0.997830,0.995899
12,2023-03-07,-0.980160,0.969970,0.951434,-0.960913
13,2023-03-08,-0.746683,0.621864,0.747876,-0.659157
14,2023-03-09,0.928006,-0.992839,-0.946337,0.986190
15,2023-03-10,0.801506,-0.994441,-0.822589,0.985471


In [166]:
import yfinance as yf
import pandas as pd

tickers_list = ['BTC-USD','DX=F', '^GSPC', '^IXIC', 'GC=F']

# Fetch data for all tickers using a for loop
data = {}
for ticker in tickers_list:
    data[ticker] = yf.Ticker(ticker).history(period='3mo', interval='1d', actions=False)['Close']

# Filter out NaN values using list comprehension
data = {ticker: close[~close.isna()] for ticker, close in data.items()}

# Merge data using inner join
merge = pd.concat(data, axis=1, join='inner')
merge.columns = [ticker for ticker, close in data.items()]

# Rename columns
merge = merge.rename(columns={"BTC-USD": "BTC", "DX=F": "DXY", "^GSPC": "S&P500", "^IXIC": "Nasdaq", "GC=F": "Gold"})
assets = [('DXY', 'DXY/BTC'), ('S&P500', 'S&P500/BTC'), ('Nasdaq', 'Nasdaq/BTC'), ('Gold', 'Gold/BTC')]

# Use a for loop to calculate the correlation for each asset pair
correlations = {}
for asset, col_name in assets:
    correlations[col_name] = merge['BTC'].rolling(3).corr(merge[asset])

# Use the assign method to add the correlations as new columns to the DataFrame
merge = merge.assign(**correlations).dropna().reset_index()

merge=merge.tail(30)


In [173]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=4, cols=1,start_cell="bottom-left",shared_xaxes=True,
    subplot_titles=("DXY/BTC", "S&P500/BTC", "Nasdaq/BTC", "Gold/BTC"))

fig.add_trace(go.Bar(x=merge['Date'], y=merge['DXY/BTC'],
                    marker=dict(color=merge['DXY/BTC'], coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(x=merge['Date'], y=merge['S&P500/BTC'],
                    marker=dict(color=merge['S&P500/BTC'], coloraxis="coloraxis")),
              2, 1)

fig.add_trace(go.Bar(x=merge['Date'], y=merge['Nasdaq/BTC'],
                    marker=dict(color=merge['Nasdaq/BTC'], coloraxis="coloraxis")),
              3, 1)

fig.add_trace(go.Bar(x=merge['Date'], y=merge['Gold/BTC'],
                    marker=dict(color=merge['Gold/BTC'], coloraxis="coloraxis")),
              4, 1)

fig.update_layout(coloraxis=dict(colorscale='Oryel'), showlegend=False)
fig.update_yaxes(nticks=3)
fig.update_xaxes(nticks=25)
fig.update_layout(height=900, width=1000, title_text="30 Day Correlation")
fig.show()

In [168]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    number = {'prefix': "$"},
    value = df['200D'].iloc[-1],
    delta = {"reference": ((df['200D'].iloc[-2])), "valueformat": ".0f"},
     title = {"text": "200 Day Moving Average"},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 0, 'column': 0}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    number = {'prefix': "$"},
    value = df['meanavge'].iloc[-1],
    delta = {"reference": ((df['meanavge'].iloc[-2])), "valueformat": ".0f"},
    title = {"text": "Meanaverage"},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 1, 'column': 0}))


fig.add_trace(go.Indicator(
    mode = "number+delta",
    number = {'prefix': "$"},
    value = df['50D'].iloc[-1],
    delta = {"reference": ((df['50D'].iloc[-2])), "valueformat": ".0f"},
    title = {"text": "50 Day Moving Average"},
    domain = {'row': 0, 'column': 1}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    number = {'prefix': "$"},
    value = df['300D'].iloc[-1],
    delta = {"reference": ((df['300D'].iloc[-2])), "valueformat": ".0f"},
    title = {"text": "300 Day Moving Average"},
    domain = {'row': 1, 'column': 1}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    number = {'prefix': "$"},
    value = df['price'].iloc[-1],
    delta = {"reference": ((df['price'].iloc[-2])), "valueformat": ".0f"},
    title = {"text": "Current Price"},
    domain = {'row': 0, 'column': 2}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    number = {'prefix': "$"},
    value = df['7D'].iloc[-1],
    delta = {"reference": ((df['7D'].iloc[-2])), "valueformat": ".0f"},
    title = {"text": "7 Day Moving Average"},
    domain = {'row': 1, 'column': 2}))

fig.update_layout(
    grid = {'rows': 2, 'columns': 3, 'pattern': "independent"},
)
fig.write_html("../static/MAs.html")
fig.show()